In [1]:
import os
import pandas as pd 
import numpy as np 

# Data preprocessors
from sklearn.utils import shuffle 
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

# ML Models (will evaluate each and choose)
from sklearn.svm import SVC # Support Vector Machine Classifier
from sklearn.naive_bayes import GaussianNB # Niave Bayes classifier
from sklearn.ensemble import RandomForestClassifier # Random Forest classifier
from sklearn.linear_model import LogisticRegression # Logistic Regression Classifier

# Deep Learning Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAvgPool1D, Dense 
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Visualization
import matplotlib.pyplot as plt

# Dumping
import joblib 

In [10]:
# Import local modules
import preprocessing_pipeline

ModuleNotFoundError: No module named 'preprocessing_pipeline'

In [4]:
# Visualization function

def plot_loss(history):
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title("Training and Validation Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    plt.tight_layout()
    plt.show()

def plot_accuracy(history):
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title("Training and Validation Accuracy")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    plt.tight_layout()
    plt.show()

In [5]:
# Data extraction function
def extract_data(DATA_PATH):
    files = [file for file in os.listdir(DATA_PATH) if os.path.isfile(os.path.join(DATA_PATH, file))]
    combined_data = pd.DataFrame()
    for file in files:
        file_path = os.path.join(DATA_PATH, file)
        df = pd.read_csv(file_path, delimiter='\t')
        combined_data = pd.concat([combined_data, df], ignore_index=True)
    return combined_data

In [6]:
# Modify the path and extract the data
data_path = '../data/'
data = extract_data(DATA_PATH=data_path)

In [7]:
data = shuffle(data)
data = data.dropna()
data.head()

,headline,content,category,date,link
2360,Erga Iiraan buufata loltootaa Ameerikaa irratt...,Erga Iiraan buufata loltootaa Ameerikaa lama...,idil_addunyaa,2020-01-08,https://www.fanabc.com/afaanoromoo/erga-iiraan...
2707,Bulchiinsoti Yuunayitid Isteet talaallii Covid...,Bulchiinsoti Yuunayitid Isteet talaallii Cov...,idil_addunyaa,2020-09-03,https://www.fanabc.com/afaanoromoo/bulchiinsot...
6239,Korporeeshinichi hidhannoo waraanaa gahaa tahe...,Raayyaan Ittisaa Hidhannoo gahaa hidhachuun b...,oduu_biyya_keessaa,2023-08-14,https://www.fanabc.com/afaanoromoo/korporeeshi...
6432,Yaahiin Innooveeshinii Diijitaalaa Afrikaa 19f...,Yaahiin Innooveeshinii Diijitaalaa Afrikaa 1...,teeknooloojii,2022-05-11,https://www.fanabc.com/afaanoromoo/yaaiin-inno...
6427,Ityoo Teelekoom Shaashamanneetti mana kitaabaa...,Itiyoo Teelekoom Magaalaa Shaashamannee mana...,teeknooloojii,2022-08-03,https://www.fanabc.com/afaanoromoo/ityoo-teele...


In [8]:
preprocessor = preprocessing_pipeline.PreprocessingPipeline()

NameError: name 'preprocessing_pipeline' is not defined

In [9]:
data['preprocessed_content'] = data['content'].apply(preprocessor.preprocess)

NameError: name 'preprocessor' is not defined